In [1]:
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer


con = sqlite3.connect('database.sqlite')

messages = pd.read_sql_query("""
SELECT Score, Summary,Text
FROM Reviews
WHERE Score != 3 order by Time
""", con)

def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

Score = messages['Score']
Score = Score.map(partition)
Text=messages['Text']
Text.head()

/home/varma/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0    this witty little book makes my son laugh at l...
1    I can remember seeing the show when it aired o...
2    Beetlejuice is a well written movie ..... ever...
3    A twist of rumplestiskin captured on film, sta...
4    A twist of rumplestiskin captured on film, sta...
Name: Text, dtype: object

# Train/Test Spliit

In [2]:
X_train,y_train=Text[:315488],Score[:315488]
X_cv,y_cv=Text[315488:420651],Score[315488:420651]
X_test,y_test=Text[420651:],Score[420651:]
print("Train Shape",X_train.shape)
print("Test Shape",X_test.shape)
print("Cross Validation Shape",X_cv.shape)

Train Shape (315488,)
Test Shape (105163,)
Cross Validation Shape (105163,)


# Converting into lowercase for train data

In [3]:
text_lst=X_train
text_lst=[item.encode('utf-8') for item in text_lst]
cX_train=[str(item).lower() for item in text_lst]
print (cX_train[0:2])

['b"this witty little book makes my son laugh at loud. i recite it in the car as we\'re driving along and he always can sing the refrain. he\'s learned about whales, india, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college"', 'b"i can remember seeing the show when it aired on television years ago, when i was a child.  my sister later bought me the lp (which i have to this day,  i\'m thirty something).i used this series of books &amp; songs when i did my  student teaching for preschoolers &amp; turned the whole school on to it.  i am now purchasing it on cd, along with the books for my children 5 &amp;  2.  the tradition lives on!"']


In [4]:
text_lst=X_cv
text_lst=[item.encode('utf-8') for item in text_lst]
cX_cv=[str(item).lower() for item in text_lst]
print (cX_cv[0:2])

['b\'i\\\'ve ordered this coffee several times & it has been excellent every time.  it\\\'s not over-roasted/burnt tasting & the flavor is very complex.  the beans are oily, black & i found no quakers (underdeveloped coffee beans that make coffee taste awful - they stick out because they do not roast/darken properly).  that\\\'s a good test of coffee beans:  empty the bag on a cookie sheet & see how many quakers you find.  a few is "acceptable" but more than 10 is pretty bad in my opinion.  if you\\\'ve always been disappointed in dark roast coffee, you may want to give this a try but if you just don\\\'t like dark roast; you will probably not like this one.<br /><br />i use the "pour over" method to brew coffee every day because at this time, there is only one non-commercial drip coffeemaker that gets water to the ideal brewing temperature of 192f to 204 f.  i use cold, filtered water, a melitta natural brown filter, manual drip coffee filter & brew directly into my stainless carafe. 

# Converting into lowercase for Test data

In [5]:
text_test=X_test
text_test=[item.encode('utf-8') for item in text_test]
cX_test=[str(item).lower() for item in text_test]
print( cX_test[0:2])

['b"i was a little hesitant when i read it had a herbal taste , i like a conventional tasting coffee. it just arrived and i immediately brewed a pot. i definitely wasn\'t disappointed, i like this much more than the usual starbuck\'s french roast"', "b'buyer beware, the worst flavored hot cocoa i have ever tasted. never again would i waste my $ for this product, i tried different machine sizes and options, the rest goes in the garbage. what a rip off with two packages.'"]


# Removing Html tags from train data

In [6]:
import re

for i in range(len(cX_train)):
    cX_train[i]=re.sub(r'<.*?>', '', cX_train[i])
print (cX_train[0:2])

['b"this witty little book makes my son laugh at loud. i recite it in the car as we\'re driving along and he always can sing the refrain. he\'s learned about whales, india, drooping roses:  i love all the new words this book  introduces and the silliness of it all.  this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college"', 'b"i can remember seeing the show when it aired on television years ago, when i was a child.  my sister later bought me the lp (which i have to this day,  i\'m thirty something).i used this series of books &amp; songs when i did my  student teaching for preschoolers &amp; turned the whole school on to it.  i am now purchasing it on cd, along with the books for my children 5 &amp;  2.  the tradition lives on!"']


In [7]:
import re

for i in range(len(cX_cv)):
    cX_cv[i]=re.sub(r'<.*?>', '', cX_cv[i])
print (cX_cv[0:2])

['b\'i\\\'ve ordered this coffee several times & it has been excellent every time.  it\\\'s not over-roasted/burnt tasting & the flavor is very complex.  the beans are oily, black & i found no quakers (underdeveloped coffee beans that make coffee taste awful - they stick out because they do not roast/darken properly).  that\\\'s a good test of coffee beans:  empty the bag on a cookie sheet & see how many quakers you find.  a few is "acceptable" but more than 10 is pretty bad in my opinion.  if you\\\'ve always been disappointed in dark roast coffee, you may want to give this a try but if you just don\\\'t like dark roast; you will probably not like this one.i use the "pour over" method to brew coffee every day because at this time, there is only one non-commercial drip coffeemaker that gets water to the ideal brewing temperature of 192f to 204 f.  i use cold, filtered water, a melitta natural brown filter, manual drip coffee filter & brew directly into my stainless carafe.  i heat the 

# Removing Html Tags from test data

In [8]:
for i in range(len(cX_test)):
    cX_test[i]=re.sub(r'<.*?>', '', cX_test[i])
print (cX_test[0:2])

['b"i was a little hesitant when i read it had a herbal taste , i like a conventional tasting coffee. it just arrived and i immediately brewed a pot. i definitely wasn\'t disappointed, i like this much more than the usual starbuck\'s french roast"', "b'buyer beware, the worst flavored hot cocoa i have ever tasted. never again would i waste my $ for this product, i tried different machine sizes and options, the rest goes in the garbage. what a rip off with two packages.'"]


# Removing Special Characters

In [9]:
for i in range(len(cX_train)):
    cX_train[i]=re.sub(r'[^a-zA-Z ]+', ' ', cX_train[i])
print (cX_train[0:2])

['b this witty little book makes my son laugh at loud  i recite it in the car as we re driving along and he always can sing the refrain  he s learned about whales  india  drooping roses   i love all the new words this book  introduces and the silliness of it all   this is a classic book i am  willing to bet my son will still be able to recite from memory when he is  in college ', 'b i can remember seeing the show when it aired on television years ago  when i was a child   my sister later bought me the lp  which i have to this day   i m thirty something i used this series of books  amp  songs when i did my  student teaching for preschoolers  amp  turned the whole school on to it   i am now purchasing it on cd  along with the books for my children    amp      the tradition lives on ']


In [10]:
for i in range(len(cX_cv)):
    cX_cv[i]=re.sub(r'[^a-zA-Z ]+', ' ', cX_cv[i])
print (cX_cv[0:2])

['b i ve ordered this coffee several times   it has been excellent every time   it s not over roasted burnt tasting   the flavor is very complex   the beans are oily  black   i found no quakers  underdeveloped coffee beans that make coffee taste awful   they stick out because they do not roast darken properly   that s a good test of coffee beans   empty the bag on a cookie sheet   see how many quakers you find   a few is  acceptable  but more than   is pretty bad in my opinion   if you ve always been disappointed in dark roast coffee  you may want to give this a try but if you just don t like dark roast  you will probably not like this one i use the  pour over  method to brew coffee every day because at this time  there is only one non commercial drip coffeemaker that gets water to the ideal brewing temperature of  f to   f   i use cold  filtered water  a melitta natural brown filter  manual drip coffee filter   brew directly into my stainless carafe   i heat the water to boiling  pour

In [11]:
for i in range(len(cX_test)):
    cX_test[i]=re.sub(r'[^a-zA-Z ]+', ' ', cX_test[i])
print (cX_test[0:2])

['b i was a little hesitant when i read it had a herbal taste   i like a conventional tasting coffee  it just arrived and i immediately brewed a pot  i definitely wasn t disappointed  i like this much more than the usual starbuck s french roast ', 'b buyer beware  the worst flavored hot cocoa i have ever tasted  never again would i waste my   for this product  i tried different machine sizes and options  the rest goes in the garbage  what a rip off with two packages ']


# Removing Stopwords

In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopword=set(stopwords.words('english'))
for i in range(len(cX_train)):
    text_stoprm=[]
    text_tokens=[]
    text_tokens=cX_train[i].split()
    for r in text_tokens:
        if not r in stopword:
            text_stoprm.append(r)
    cX_train[i]=' '.join(text_stoprm)
    
print (cX_train[0:2])

['b witty little book makes son laugh loud recite car driving along always sing refrain learned whales india drooping roses love new words book introduces silliness classic book willing bet son still able recite memory college', 'b remember seeing show aired television years ago child sister later bought lp day thirty something used series books amp songs student teaching preschoolers amp turned whole school purchasing cd along books children amp tradition lives']


In [13]:
for i in range(len(cX_cv)):
    text_stoprm=[]
    text_tokens=[]
    text_tokens=cX_cv[i].split()
    for r in text_tokens:
        if not r in stopword:
            text_stoprm.append(r)
    cX_cv[i]=' '.join(text_stoprm)
    
print (cX_cv[0:2])

['b ordered coffee several times excellent every time roasted burnt tasting flavor complex beans oily black found quakers underdeveloped coffee beans make coffee taste awful stick roast darken properly good test coffee beans empty bag cookie sheet see many quakers find acceptable pretty bad opinion always disappointed dark roast coffee may want give try like dark roast probably like one use pour method brew coffee every day time one non commercial drip coffeemaker gets water ideal brewing temperature f f use cold filtered water melitta natural brown filter manual drip coffee filter brew directly stainless carafe heat water boiling pour tablespoons grounds saturate plump beans wait minute flood grounds rest water lot crema beautiful reddish brown foam develops top beans smells tastes wonderful brew directly carafe using coffee cone pour method rsvp manual drip coffee filter cone carafes thermosi use grinder bodum bistro electric burr coffee grinder blacki use water kettle heat water ost

In [14]:
for i in range(len(cX_test)):
    text_stoprm=[]
    text_tokens=[]
    text_tokens=cX_test[i].split()
    for r in text_tokens:
        if not r in stopword:
            text_stoprm.append(r)
    cX_test[i]=' '.join(text_stoprm)
    
print( cX_test[0:2])

['b little hesitant read herbal taste like conventional tasting coffee arrived immediately brewed pot definitely disappointed like much usual starbuck french roast', 'b buyer beware worst flavored hot cocoa ever tasted never would waste product tried different machine sizes options rest goes garbage rip two packages']


# Stemming Data

In [15]:
from nltk.stem.snowball import SnowballStemmer
stemmer =SnowballStemmer('english')

for i in range(len(cX_train)):
    text_stem=[]
    text_tokens=[]
    text_tokens=cX_train[i].split()
    for r in text_tokens:
        text_stem.append(str(stemmer.stem(r)))
    cX_train[i]=' '.join(text_stem)
    


In [16]:

for i in range(len(cX_cv)):
    text_stem=[]
    text_tokens=[]
    text_tokens=cX_cv[i].split()
    for r in text_tokens:
        text_stem.append(str(stemmer.stem(r)))
    cX_cv[i]=' '.join(text_stem)
    

In [17]:
from nltk.stem.snowball import SnowballStemmer
stemmer =SnowballStemmer('english')

for i in range(len(cX_test)):
    text_stem=[]
    text_tokens=[]
    text_tokens=cX_test[i].split()
    for r in text_tokens:
        text_stem.append(str(stemmer.stem(r)))
    cX_test[i]=' '.join(text_stem)
    


# Tokenization

In [18]:
from nltk.tokenize import word_tokenize
tokenized_sents_train = [word_tokenize(i) for i in cX_train]
print(tokenized_sents_train[:2])

[['b', 'witti', 'littl', 'book', 'make', 'son', 'laugh', 'loud', 'recit', 'car', 'drive', 'along', 'alway', 'sing', 'refrain', 'learn', 'whale', 'india', 'droop', 'rose', 'love', 'new', 'word', 'book', 'introduc', 'silli', 'classic', 'book', 'will', 'bet', 'son', 'still', 'abl', 'recit', 'memori', 'colleg'], ['b', 'rememb', 'see', 'show', 'air', 'televis', 'year', 'ago', 'child', 'sister', 'later', 'bought', 'lp', 'day', 'thirti', 'someth', 'use', 'seri', 'book', 'amp', 'song', 'student', 'teach', 'preschool', 'amp', 'turn', 'whole', 'school', 'purchas', 'cd', 'along', 'book', 'children', 'amp', 'tradit', 'live']]


In [19]:
from nltk.tokenize import word_tokenize
tokenized_sents_cv = [word_tokenize(i) for i in cX_cv]
print(tokenized_sents_cv[:2])

[['b', 'order', 'coffe', 'sever', 'time', 'excel', 'everi', 'time', 'roast', 'burnt', 'tast', 'flavor', 'complex', 'bean', 'oili', 'black', 'found', 'quaker', 'underdevelop', 'coffe', 'bean', 'make', 'coffe', 'tast', 'aw', 'stick', 'roast', 'darken', 'proper', 'good', 'test', 'coffe', 'bean', 'empti', 'bag', 'cooki', 'sheet', 'see', 'mani', 'quaker', 'find', 'accept', 'pretti', 'bad', 'opinion', 'alway', 'disappoint', 'dark', 'roast', 'coffe', 'may', 'want', 'give', 'tri', 'like', 'dark', 'roast', 'probabl', 'like', 'one', 'use', 'pour', 'method', 'brew', 'coffe', 'everi', 'day', 'time', 'one', 'non', 'commerci', 'drip', 'coffeemak', 'get', 'water', 'ideal', 'brew', 'temperatur', 'f', 'f', 'use', 'cold', 'filter', 'water', 'melitta', 'natur', 'brown', 'filter', 'manual', 'drip', 'coffe', 'filter', 'brew', 'direct', 'stainless', 'caraf', 'heat', 'water', 'boil', 'pour', 'tablespoon', 'ground', 'satur', 'plump', 'bean', 'wait', 'minut', 'flood', 'ground', 'rest', 'water', 'lot', 'crema',

In [20]:
from nltk.tokenize import word_tokenize
tokenized_sents_test = [word_tokenize(i) for i in cX_test]
print(tokenized_sents_test[:2])

[['b', 'littl', 'hesit', 'read', 'herbal', 'tast', 'like', 'convent', 'tast', 'coffe', 'arriv', 'immedi', 'brew', 'pot', 'definit', 'disappoint', 'like', 'much', 'usual', 'starbuck', 'french', 'roast'], ['b', 'buyer', 'bewar', 'worst', 'flavor', 'hot', 'cocoa', 'ever', 'tast', 'never', 'would', 'wast', 'product', 'tri', 'differ', 'machin', 'size', 'option', 'rest', 'goe', 'garbag', 'rip', 'two', 'packag']]


# Word2Vec 

In [37]:
from gensim.models import Word2Vec

model_train=Word2Vec(tokenized_sents_train,workers=8,min_count=1,size=50)
print(model_train)
words=list(model_train.wv.vocab)
print(words)
print(model_train['loud'])

Word2Vec(vocab=65325, size=50, alpha=0.025)
['b', 'witti', 'littl', 'book', 'make', 'son', 'laugh', 'loud', 'recit', 'car', 'drive', 'along', 'alway', 'sing', 'refrain', 'learn', 'whale', 'india', 'droop', 'rose', 'love', 'new', 'word', 'introduc', 'silli', 'classic', 'will', 'bet', 'still', 'abl', 'memori', 'colleg', 'rememb', 'see', 'show', 'air', 'televis', 'year', 'ago', 'child', 'sister', 'later', 'bought', 'lp', 'day', 'thirti', 'someth', 'use', 'seri', 'amp', 'song', 'student', 'teach', 'preschool', 'turn', 'whole', 'school', 'purchas', 'cd', 'children', 'tradit', 'live', 'beetlejuic', 'well', 'written', 'movi', 'everyth', 'excel', 'act', 'special', 'effect', 'delight', 'chose', 'view', 'twist', 'rumplestiskin', 'captur', 'film', 'star', 'michael', 'keaton', 'geena', 'davi', 'prime', 'tim', 'burton', 'masterpiec', 'rumbl', 'absurd', 'wonder', 'pace', 'point', 'dull', 'moment', 'funni', 'hilari', 'wacki', 'great', 'help', 'think', 'one', 'best', 'ever', 'made', 'sure', 'agre', 'g

/home/varma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [38]:
from gensim.models import Word2Vec

model_cv=Word2Vec(tokenized_sents_cv,workers=8,min_count=1,size=50)
print(model_cv)
words=list(model_cv.wv.vocab)
print(words)
print(model_cv['pour'])

Word2Vec(vocab=36413, size=50, alpha=0.025)
['b', 'order', 'coffe', 'sever', 'time', 'excel', 'everi', 'roast', 'burnt', 'tast', 'flavor', 'complex', 'bean', 'oili', 'black', 'found', 'quaker', 'underdevelop', 'make', 'aw', 'stick', 'darken', 'proper', 'good', 'test', 'empti', 'bag', 'cooki', 'sheet', 'see', 'mani', 'find', 'accept', 'pretti', 'bad', 'opinion', 'alway', 'disappoint', 'dark', 'may', 'want', 'give', 'tri', 'like', 'probabl', 'one', 'use', 'pour', 'method', 'brew', 'day', 'non', 'commerci', 'drip', 'coffeemak', 'get', 'water', 'ideal', 'temperatur', 'f', 'cold', 'filter', 'melitta', 'natur', 'brown', 'manual', 'direct', 'stainless', 'caraf', 'heat', 'boil', 'tablespoon', 'ground', 'satur', 'plump', 'wait', 'minut', 'flood', 'rest', 'lot', 'crema', 'beauti', 'reddish', 'foam', 'develop', 'top', 'smell', 'wonder', 'cone', 'rsvp', 'thermosi', 'grinder', 'bodum', 'bistro', 'electr', 'burr', 'blacki', 'kettl', 'oster', 'steel', 'liter', 'kettlei', 'count', 'love', 'stuff', 'pe

/home/varma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [39]:
from gensim.models import Word2Vec

model_test=Word2Vec(tokenized_sents_test,workers=8,min_count=1,size=50)
print(model_test)
words=list(model_test.wv.vocab)
print(words)
print(model_test['rip'])

Word2Vec(vocab=37219, size=50, alpha=0.025)
['b', 'littl', 'hesit', 'read', 'herbal', 'tast', 'like', 'convent', 'coffe', 'arriv', 'immedi', 'brew', 'pot', 'definit', 'disappoint', 'much', 'usual', 'starbuck', 'french', 'roast', 'buyer', 'bewar', 'worst', 'flavor', 'hot', 'cocoa', 'ever', 'never', 'would', 'wast', 'product', 'tri', 'differ', 'machin', 'size', 'option', 'rest', 'goe', 'garbag', 'rip', 'two', 'packag', 'absolut', 'grew', 'chef', 'boyarde', 'ravioli', 'clearest', 'memori', 'sit', 'granni', 'hous', 'eat', 'lunch', 'love', 'well', 'kid', 'give', 'everi', 'time', 'good', 'close', 'reminisc', 'enough', 'use', 'sooooooooo', 'day', 'ago', 'grab', 'campbel', 'raviolio', 'seen', 'dang', 'prefer', 'one', 'clump', 'togeth', 'cook', 'without', 'fall', 'apart', 'realli', 'lot', 'better', 'ran', 'got', 'three', 'can', 'next', 'shop', 'trip', 'think', 'lost', 'touch', 'campbell', 'chanc', 'sheri', 'gill', 'writer', 'problem', 'order', 'case', 'half', 'best', 'kettl', 'chip', 'get', 'wo

/home/varma/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


# Average Word2Vec

In [40]:
sentece_vec_train=[]
for sent in tokenized_sents_train:
    sent_vec=np.zeros(50)
    cnt_words=0
    for word in sent:
        vec=model_train.wv[word]
        sent_vec+=vec
        cnt_words+=1
        
    sent_vec/=cnt_words
    sentece_vec_train.append(sent_vec)
    
print(len(sentece_vec_train))
print(len(sentece_vec_train[0]))

315488
50


In [41]:
sentece_vec_cv=[]
for sent in tokenized_sents_cv:
    sent_vec=np.zeros(50)
    cnt_words=0
    for word in sent:
        vec=model_cv.wv[word]
        sent_vec+=vec
        cnt_words+=1
        
    sent_vec/=cnt_words
    sentece_vec_cv.append(sent_vec)
    
print(len(sentece_vec_cv))
print(len(sentece_vec_cv[0]))

105163
50


In [42]:
sentece_vec_test=[]
for sent in tokenized_sents_test:
    sent_vec=np.zeros(50)
    cnt_words=0
    for word in sent:
        vec=model_test.wv[word]
        sent_vec+=vec
        cnt_words+=1
        
    sent_vec/=cnt_words
    sentece_vec_test.append(sent_vec)
    
print(len(sentece_vec_test))
print(len(sentece_vec_test[0]))

105163
50


# Modelling on AVG Word2Vec

# Logistic Regression 

In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
alpha=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]

maxval=0
bestalpha=0
for i in alpha:
    logistic_clf=LogisticRegression(C=i)
    logistic_clf.fit(sentece_vec_train,y_train)
    cvpredicted=logistic_clf.predict(sentece_vec_cv)
    score=accuracy_score(y_cv,cvpredicted)
    print('for alpha',i,'accuracy score is',score)
    if(maxval<score):
        maxval=score
        bestalpha=i
        
print('-'*100)
print('the best alpha is ',bestalpha,'and score on CV is',maxval)

print('-'*100)

#for test data
clf=LogisticRegression(C=bestalpha)
clf.fit(sentece_vec_train,y_train)

trainpredicted=clf.predict(sentece_vec_train)
score=accuracy_score(y_train,trainpredicted)
print('train accuracy is',score)

cvpredicted=clf.predict(sentece_vec_cv)
score=accuracy_score(y_cv,cvpredicted)
print('cv accuracy',score)

testpredicted=clf.predict(sentece_vec_test)
score=accuracy_score(y_test,testpredicted)
print('test accuracy',score)


for alpha 1e-05 accuracy score is 0.8238829246027596
for alpha 0.0001 accuracy score is 0.8366630849253064
for alpha 0.001 accuracy score is 0.8358928520487243
for alpha 0.01 accuracy score is 0.8326597757766515
for alpha 0.1 accuracy score is 0.8273917632627446
for alpha 1 accuracy score is 0.8236737255498607
for alpha 10 accuracy score is 0.8232363093483449
for alpha 100 accuracy score is 0.8232743455397811
for alpha 1000 accuracy score is 0.8232743455397811
----------------------------------------------------------------------------------------------------
the best alpha is  0.0001 and score on CV is 0.8366630849253064
----------------------------------------------------------------------------------------------------
train accuracy is 0.8786863525712547
cv accuracy 0.8366630849253064
test accuracy 0.839563344522313


In [44]:
from sklearn.linear_model import SGDClassifier

alpha=[0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]

maxval=0
bestalpha=0
for i in alpha:
    svm_clf=SGDClassifier(alpha=i,loss='hinge')
    svm_clf.fit(sentece_vec_train,y_train)
    cvpredicted=svm_clf.predict(sentece_vec_cv)
    score=accuracy_score(y_cv,cvpredicted)
    print('for alpha',i,'accuracy score is',score)
    if(maxval<score):
        maxval=score
        bestalpha=i
        
print('-'*100)
print('the best alpha is ',bestalpha,'and score on CV is',maxval)

print('-'*100)

#for test data
clf=SGDClassifier(alpha=bestalpha,loss='hinge')
clf.fit(sentece_vec_train,y_train)

trainpredicted=clf.predict(sentece_vec_train)
score=accuracy_score(y_train,trainpredicted)
print('train accuracy is',score)

cvpredicted=clf.predict(sentece_vec_cv)
score=accuracy_score(y_cv,cvpredicted)
print('cv accuracy',score)

testpredicted=clf.predict(sentece_vec_test)
score=accuracy_score(y_test,testpredicted)
print('test accuracy',score)


/home/varma/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


for alpha 1e-05 accuracy score is 0.8072135637058662
for alpha 0.0001 accuracy score is 0.8318134705171971
for alpha 0.001 accuracy score is 0.8388026206935899
for alpha 0.01 accuracy score is 0.8318895429000694
for alpha 0.1 accuracy score is 0.822237859323146
for alpha 1 accuracy score is 0.822237859323146
for alpha 10 accuracy score is 0.822237859323146
for alpha 100 accuracy score is 0.17776214067685403
for alpha 1000 accuracy score is 0.822237859323146
----------------------------------------------------------------------------------------------------
the best alpha is  0.001 and score on CV is 0.8388026206935899
----------------------------------------------------------------------------------------------------
train accuracy is 0.897932726442844
cv accuracy 0.8392210187993876
test accuracy 0.8489582838070424


In [49]:
from sklearn.ensemble import RandomForestClassifier

alpha=[10,50,100,500,1000]
maxval=0
bestalpha=0
for i in alpha:
    r_clf=RandomForestClassifier(n_estimators=i,n_jobs=-1)
    r_clf.fit(sentece_vec_train,y_train)
    cvpredicted=r_clf.predict(sentece_vec_cv)
    score=accuracy_score(y_cv,cvpredicted)
    print('for alpha',i,'accuracy score is',score)
    if(maxval<score):
        maxval=score
        bestalpha=i
        
print('-'*100)
print('the best alpha is ',bestalpha,'and score on CV is',maxval)

print('-'*100)


clf=RandomForestClassifier(n_estimators=bestalpha,n_jobs=-1)
clf.fit(sentece_vec_train,y_train)

trainpredicted=clf.predict(sentece_vec_train)
score=accuracy_score(y_train,trainpredicted)
print('train accuracy is',score)

cvpredicted=clf.predict(sentece_vec_cv)
score=accuracy_score(y_cv,cvpredicted)
print('cv accuracy',score)

testpredicted=clf.predict(sentece_vec_test)
score=accuracy_score(y_test,testpredicted)
print('test accuracy',score)

    
    

for alpha 10 accuracy score is 0.8226372393332255
for alpha 50 accuracy score is 0.8248718655800995
for alpha 100 accuracy score is 0.8238163612677463
for alpha 500 accuracy score is 0.8227988931468292
for alpha 1000 accuracy score is 0.8228464383861244
----------------------------------------------------------------------------------------------------
the best alpha is  50 and score on CV is 0.8248718655800995
----------------------------------------------------------------------------------------------------
train accuracy is 0.9999302667613348
cv accuracy 0.8248908836758175
test accuracy 0.830776984300562
